1. pip Kurulumu

In [ ]:
!pip install tensorflow
!pip install gym
!pip install keras
!pip install keras_rl2

In [ ]:
!pip install gym[all]

In [ ]:
!pip install gym[atari]



2. Oyun Testi

In [ ]:
import gym,random
env=gym.make("SpaceInvaders-v0")
h,w,channels=env.observation_space.shape
actions=env.action_space.n

In [ ]:
episodes=5
for episode in range(1,episodes+1):
    state=env.reset()
    done=False
    score=0 

    while not done:
        env.render()
        action=random.choice([0,1,2,3,4,5])
        n_state,reward,done,info=env.step(action)
        score+=reward
    print(f"Episode: {episode}  Score: {score}")
env.close()    

3. Model Oluşturma

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Convolution2D
from tensorflow.keras.optimizers import Adam

In [ ]:
def build_model(h,w,channels,actions):
    model=Sequential()
    model.add(Convolution2D(32,(8,8),strides=(4,4),activation='relu',input_shape=(3,h,w,channels)))
    model.add(Convolution2D(64,(4,4),strides=(2,2),activation='relu'))
    model.add(Convolution2D(64,(3,3),activation='relu'))
    model.add(Flatten())
    model.add(Dense(512,activation='relu'))
    model.add(Dense(256,activation='relu'))
    model.add(Dense(actions,activation='linear'))
    return model

In [ ]:
model=build_model(h,w,channels,actions)
model.summary()

4. Agent Oluşturma

In [ ]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy,EpsGreedyQPolicy

In [ ]:
def build_agent(model,actions):
    policy=LinearAnnealedPolicy(EpsGreedyQPolicy(),attr='eps',
    value_max=1.,value_min=.1,value_test=.2,nb_steps=10000)

    memory=SequentialMemory(limit=1000,window_length=3)

    dqn=DQNAgent(model=model,memory=memory,policy=policy,
    enable_dueling_network=True,dueling_type='avg',
    nb_actions=actions,nb_steps_warmup=10000)

    return dqn

In [ ]:
from tensorflow.python.framework.ops import disable_eager_execution

disable_eager_execution()

In [ ]:
dqn=build_agent(model,actions)
dqn.compile(Adam(lr=1e-4))
dqn.fit(env,nb_steps=5000,visualize=False,verbose=2)

In [ ]:
scores=dqn.test(env,nb_episodes=5,visualize=True)
print(np.mean(scores.history['episode_reward']))

In [ ]:
dqn.save_weights('SavedWeights/100k/dqn_weights.h5f')

In [ ]:
dqn.load_weights('SavedWeights/100k/dqn_weights.h5f')